# 광고평가 서술형 응답을 대분류/소분류 하면서   긍정/부정 토팍(=명사)추출하자
### engeneering 예외처리 추가
1. Q15 3글자 미만의 경우 분류하지 않음(Not enough data) 로 대체


In [29]:
import os
import anthropic
import time
import json
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from tqdm import tqdm

# .env 파일에서 API 키 로드
load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=anthropic_api_key,
)

MAX_DAILY_REQUESTS = 5000  # 일일 최대 요청 수 설정 (예시 값)
request_count = 0
start_time = datetime.now()




In [31]:
def analyze_responses(responses):
    prompt = f"""<context>You need to classify the survey response data of outdoor clothing ads into minor and major classifications, and extract positive and negative keywords for ad evaluation analysis from the response text. </context>

    <response text>
    {responses}
    </response text>

    <instruction>
    1. The response text classification is classified by reference to the <Classification> as a whole. 
    2. <Classification> consists of [Major Classification],[Minor Classification],[Reason for Classification]
    3. when classifying response text, classify the major classification first, and then classify the minor classification corresponding to the major classification.
    4. Extract positive and negative topics from the response text for ad evaluation analysis.
    5. Positive and negative topics should be extracted by noun.
    6. If there are multiple extracted keywords, they are separated by |.
    6. The output should be a CSV in the following format
        Major Classification, Minor Classification, Positive topics, Negative topics
    7. All output languages should be in your language
    8. Please print out the only result without the title
    8. Please Create accurate and reliable results for insightful ad survey analysis
     Note: Due to the CSV format, commas (,) should only be used as field separators, and if there are commas in the content, please change them to spaces
    </instruction>

    <Classification>
[Major Classification],[Minor Classification],[Reason for Classification]
감성 소구,긍정적,광고의 감성적 요소가 마음에 든다는 평가
감성 소구,부정적,감성적 요소가 부족하거나 어울리지 않는다는 평가
감성 소구,따뜻함,따뜻하고 포근한 감성을 전달함
감성 소구,고급스러움,고급스럽고 세련된 이미지를 전달함
감성 소구,긍정적,따뜻하고 편안한 느낌을 줌 공감이 됨
감성 소구,부정적,감정적 연결이 부족함 공감이 되지 않음
광고 구성,긍정적 평가,광고 구성이 좋다 전개가 자연스럽다 등의 표현
광고 구성,부정적 평가,광고 구성이 어색하다 전개가 부자연스럽다 등의 표현
광고 임팩트,강한 임팩트,인상적이다 기억에 남는다 등의 표현
광고 임팩트,약한 임팩트,평범하다 특별한 점이 없다 등의 표현
광고 전반,신선함,기존 광고와 다른 새로운 느낌을 줌
광고 전반,평범함,다른 아웃도어 광고와 큰 차이가 없음
광고 전반,몰입도,광고에 집중되고 끝까지 보게 됨
광고 전반,기억에 남지 않음,특별히 인상적이거나 기억에 남는 점이 없음
구매 의도,긍정적,광고를 보고 제품을 구매하고 싶다는 의견
구매 의도,부정적,광고를 봐도 구매 의사가 생기지 않는다는 의견
기능성 강조,긍정적 평가,제품의 기능성이 잘 표현되었다는 평가
기능성 강조,부정적 평가,기능성 표현이 부족하거나 과장되었다는 평가
기능성 강조,보온성,추위를 잘 막아줄 것 같은 느낌을 줌
기능성 강조,기술력,첨단 기술이 적용된 제품으로 느껴짐
기억력,높음,인상 깊고 기억에 남는다는 평가
기억력,낮음,별로 기억에 남지 않는다는 평가
독창성,높음,새롭고 참신한 광고라는 평가
독창성,낮음,식상하고 평범한 광고라는 평가
메시지/카피,이해도 높음,광고 메시지가 명확하고 이해하기 쉽다는 평가
메시지/카피,이해도 낮음,광고 내용이 이해하기 어렵다는 평가
메시지/카피,공감도 높음,광고 메시지에 공감한다는 평가
메시지/카피,공감도 낮음,광고 메시지에 공감하지 못한다는 평가
메시지/카피,독창성,카피가 독특하고 인상적임
메시지/카피,공감도,광고 메시지에 공감이 됨
메시지/카피,긍정적 평가,제품의 특징을 잘 전달함 기억에 남는 문구
메시지/카피,부정적 평가,이해하기 어려움 특별한 점이 없음
모델,호감도 높음,모델이 마음에 든다 예쁘다 잘 어울린다 등의 긍정적 평가
모델,호감도 낮음,모델이 광고나 브랜드와 어울리지 않음
모델,인지도,유명 모델이라 기억에 남음
모델,긍정적 평가,모델이 광고와 잘 어울림 호감도 높음 신선하고 젊은 이미지
모델,부정적 평가,모델이 제품과 어울리지 않음 모델에만 집중됨
비주얼,배경,눈 덮인 배경이 아름답고 겨울 분위기와 잘 어울림
비주얼,색감,전체적인 색감이 부드럽고 따뜻해 보임
비주얼,영상미,영상의 구성과 편집이 세련되고 아름다움
비주얼,배경,배경이 좋다 자연 배경이 마음에 든다 등의 표현
비주얼,전체적 분위기,분위기가 좋다 시원해 보인다 등의 표현
시각적 요소,배경,자연 풍경이나 도시 배경이 마음에 든다는 평가
시각적 요소,색감,밝고 경쾌한 색감이 좋다는 평가
시각적 요소,영상미,전반적인 영상의 퀄리티나 미적 요소에 대한 평가
시각적 요소,긍정적 평가,배경이 아름답고 제품과 잘 어울림 전체적인 분위기가 좋음
시각적 요소,부정적 평가,너무 어둡거나 밋밋함 특별한 점이 없음
신뢰성,긍정적 평가,신뢰가 간다 믿음이 간다 등의 표현
신뢰성,부정적 평가,신뢰가 가지 않는다 믿음이 가지 않는다 등의 표현
음악/사운드,긍정적 평가,경쾌하고 좋은 음악이라는 평가
음악/사운드,부정적 평가,음악이 어울리지 않거나 별로라는 평가
음악/사운드,어울림,배경 음악이 광고 분위기와 잘 어울림
음악/사운드,기억에 남음,음악이 인상적이고 기억에 남음
전문성,긍정적 평가,전문적으로 보인다 등의 표현
전문성,부정적 평가,전문성이 부족해 보인다 등의 표현
전반적 평가,긍정적,전체적으로 좋은 광고라는 평가
전반적 평가,부정적,전체적으로 아쉬운 점이 많다는 평가
전반적 평가,중립적,특별히 좋거나 나쁘지 않다는 평가
제품/브랜드,기능성 강조,제품의 기능이나 특징이 잘 드러난다는 평가
제품/브랜드,디자인,제품 디자인에 대한 긍정적 또는 부정적 평가
제품/브랜드,브랜드 이미지,브랜드에 대한 전반적인 이미지 평가
제품/브랜드,긍정적 평가,제품의 기능성이 잘 드러남 브랜드 이미지와 잘 맞음
제품/브랜드,부정적 평가,제품이 눈에 띄지 않음 브랜드와 어울리지 않음
창의성,긍정적,새롭고 참신한 광고라고 느낌
창의성,부정적,식상하고 진부한 느낌
추천 의도,긍정적,다른 사람에게 추천하고 싶다는 의견
추천 의도,부정적,다른 사람에게 추천하고 싶지 않다는 의견
타겟 적합성,긍정적,광고가 타겟층에 잘 맞는다는 평가
타겟 적합성,부정적,광고가 타겟층과 맞지 않는다는 평가
타겟 적합성,젊은층 타겟,젊은 층을 겨냥한 광고로 느껴짐
타겟 적합성,일반 소비자 타겟,다양한 연령층이 사용할 수 있는 제품으로 느껴짐
기타, , 
없음/모름,  , 
    </Classification>
    """
        
    try:
        message = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=4000, 
            temperature=0.4,
            system="From now on, you are a video ad evaluation survey analytics expert, and you need to analyse the narrative ad evaluations that have been answered.",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        }
                    ]
                }
            ]
        )
        return message.content
    except anthropic.RateLimitError as e:
        print(f"요청 제한 초과: {e}")
        raise
    except anthropic.InternalServerError as e:
        print(f"내부 서버 오류 발생: {e}")
        raise
    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")
        raise


In [ ]:
def read_csv(file_path):
    try:
        # CSV 파일을 읽을 때 on_bad_lines='warn'을 사용하여 문제가 있는 행을 경고로 처리
        return pd.read_csv(file_path, encoding='utf-8-sig', quoting=3, on_bad_lines='warn')
    except Exception as e:
        print(f"CSV 파일 읽기 중 오류 발생: {e}")
        # 오류가 발생한 경우, 더 관대한 옵션으로 다시 시도
        return pd.read_csv(file_path, encoding='utf-8-sig', quoting=3, on_bad_lines='warn', sep=None, engine='python')

@retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=4, max=60),
    retry=retry_if_exception_type((anthropic.InternalServerError, anthropic.RateLimitError))
)


def process_response(analysis):
    if isinstance(analysis, list) and len(analysis) > 0:
        text = analysis[0].text if hasattr(analysis[0], 'text') else str(analysis[0])
    else:
        text = str(analysis)
    
    parts = text.split(',')
    if len(parts) == 4:
        return {
            'Major Classification': parts[0].strip(),
            'Minor Classification': parts[1].strip(),
            'Positive topics': parts[2].strip(),
            'Negative topics': parts[3].strip()
        }
    return None

def process_batch(batch, start_index):
    results = []
    for index, row in batch.iterrows():
        result = {
            'ID': row['ID'],
            'wave': row['wave'],
            'IDX': row['IDX'],
            'Q1': row['Q1'],
            'Q2': row['Q2'],
            'Q2_R': row['Q2_R'],
            'ad_type': row['ad_type'],
            'Likert': row['Likert'],
            'Q15': row['Q15']
        }
        
        try:
            response = row['Q15']

            if len(response.strip()) <= 3: 
                result.update({
                    'Major Classification': 'Not enough data',
                    'Minor Classification': 'Not enough data',
                    'Positive topics': '',
                    'Negative topics': ''
                })
            else:
                analysis = analyze_responses(response)
                analysis_result = process_response(analysis)
            
                if analysis_result:
                    result.update(analysis_result)
                else:
                    result.update({
                        'Major Classification': '',
                        'Minor Classification': '',
                        'Positive topics': '',
                        'Negative topics': ''
                    })
        except Exception as e:
            print(f"행 {start_index + index + 1} 처리 중 오류 발생: {e}")
            result.update({
                'Major Classification': 'Error',
                'Minor Classification': 'Error',
                'Positive topics': '',
                'Negative topics': ''
            })
        
        results.append(result)
        
        #print(f"Processed row {start_index + index + 1}")
        
        time.sleep(2)  # 각 요청 사이에 10초 대기
    
    return results

def save_checkpoint(current_index, all_results):
    checkpoint = {
        'current_index': current_index,
        'results': all_results
    }
    with open('checkpoint.json', 'w', encoding='utf-8') as f:
        json.dump(checkpoint, f, ensure_ascii=False, indent=2)

def load_checkpoint():
    if os.path.exists('checkpoint.json'):
        with open('checkpoint.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    return None

def main():
    #input_file = 'data_ad_total_2401.csv'  #24년 1wave 본파일 
    #input_file = 'data_ad_test.csv'  #테스트 파일 

    input_file = 'data_ad_total_2401_add.csv'  #부족분 재실행 파일
        
    output_file = "[labeling]" + input_file
    batch_size = 10  # 배치 크기를 더 줄임
    
    df = read_csv(input_file)
    
    #데이터의 크기만 확인
    print(df.shape)

    #df 전체 데이터중 처음부터 500개만 사용
    df = df.head(3)
    
    #df 데이터 중 1200번 부터 2000번 사용
    #df = df.iloc[2140:2501]
    

    #df 데이터 중 1000번 부터 끝까지 사용    
    #df = df.iloc[1000:]
    
    
    print(df.shape)
    
    checkpoint = load_checkpoint()
    if checkpoint:
        start_index = checkpoint['current_index']
        all_results = checkpoint['results']
        print(f"체크포인트에서 재개: 인덱스 {start_index}")
    else:
        start_index = 0
        all_results = []
    
    total_batches = (len(df) - start_index + batch_size - 1) // batch_size
    
    with tqdm(total=total_batches, desc="Overall Progress") as pbar:
        for i in range(start_index, len(df), batch_size):
            batch = df.iloc[i:i+batch_size]
            results = process_batch(batch, i)
            all_results.extend(results)
            
            # 중간 결과 및 체크포인트 저장
            result_df = pd.DataFrame(all_results)
            result_df.to_csv(output_file, index=False, encoding='utf-8-sig')
            save_checkpoint(i + batch_size, all_results)
            print(f"중간 결과 및 체크포인트가 저장되었습니다.")
            
            pbar.update(1) 
            time.sleep(5)  # 각 배치 사이에 20초 대기
    
    print(f"분석이 완료되었습니다. 최종 결과가 {output_file}에 저장되었습니다.")

if __name__ == "__main__":
    main()

